# Imports

In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta

# Risk Analyzer Class 
using dummy variables

In [3]:

#Dummy variables
stock_ticker = '1120.SR' 
market_ticker = '^TASI.SR'


class RiskAnalyzer:

    # Initizialize with stock and market tickers, and lookbback period (set to two years by default)
    def __init__(self, stock_ticker, market_ticker , lookback_days=2*365):
        self.stock_ticker = stock_ticker
        self.market_ticker = market_ticker
        self.lookback_days = lookback_days
        self.stock_returns = None
        self.market_returns = None
        self.start_date = None
        self.end_date = None
    
    # Fetch historical prices and compute daily returns
    def get_returns(self, ticker):
        self.end_date = datetime.now().date()
        self.start_date = self.end_date - timedelta(days = self.lookback_days)
        prices = yf.download(ticker, start=self.start_date, end=self.end_date)['Adj Close']
        returns = prices.pct_change().dropna()
        return returns
    
    # Load returns for stock and market index
    def load_returns(self):
        stock_returns = self.get_returns(self.stock_ticker)
        market_returns = self.get_returns(self.market_ticker)

        # Align both returns to the same dates
        aligned_returns = stock_returns.align(market_returns, join= 'inner')
        self.stock_returns = aligned_returns[0]
        self.market_returns = aligned_returns[1]

    # Calculate beta (stock volatility relative to the market)
    def calculate_beta(self):
        self.load_returns()
        beta = np.cov(self.stock_returns, self.market_returns)[0, 1] / np.var(self.market_returns)
        return beta
    
    # Calculate systematic risk (market-related risk)
    def calculate_systematic_risk(self):
        beta = self.calculate_beta()
        systematic_risk = beta**2 * np.var(self.market_returns)
        return systematic_risk

    # Calculate unsystematic risk (stock-specific risk)
    def calculate_unsystematic_risk(self):
        self.load_returns()
        total_risk = np.var(self.stock_returns)
        systematic_risk = self.calculate_systematic_risk()
        unsystematic_risk = total_risk - systematic_risk
        return unsystematic_risk


# Generate Risk Assessment Report

In [4]:


# Generate risk assessment report
def generate_report():
    analyzer = RiskAnalyzer(stock_ticker, market_ticker )
    beta = analyzer.calculate_beta()
    systematic_risk = analyzer.calculate_systematic_risk()
    unsystematic_risk = analyzer.calculate_unsystematic_risk()
    start_date = analyzer.start_date
    end_date = analyzer.end_date

    report = (
        f"""
    Risk Assessment Report
    ----------------------
    Date: {datetime.now().strftime('%Y-%m-%d')}
    
    Stock: {stock_ticker}
    Market Index: {market_ticker}
    Period: From {start_date} to {end_date}
    
    Beta: {beta:.4f}
    Systematic Risk: {systematic_risk:.7f}
    Unsystematic Risk: {unsystematic_risk:.7f}
    
    Summary:
    - The beta value of {beta:.4f} indicates that the stock is {'more' if beta > 1 else 'less'} volatile compared to the market.
    - Systematic risk represents the risk that cannot be diversified away, influenced by market factors.
    - Unsystematic risk, at {unsystematic_risk:.4f}, is the portion of risk specific to {stock_ticker}, which can be mitigated through diversification.
    """
    )
    return report


report = generate_report()
print(report)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


    Risk Assessment Report
    ----------------------
    Date: 2024-08-25
    
    Stock: 1120.SR
    Market Index: ^TASI.SR
    Period: From 2022-08-26 to 2024-08-25
    
    Beta: 1.1271
    Systematic Risk: 0.0000814
    Unsystematic Risk: 0.0000855
    
    Summary:
    - The beta value of 1.1271 indicates that the stock is more volatile compared to the market.
    - Systematic risk represents the risk that cannot be diversified away, influenced by market factors.
    - Unsystematic risk, at 0.0001, is the portion of risk specific to 1120.SR, which can be mitigated through diversification.
    
